In [ ]:
#imported train_values, test_values and train_label 
#merged train_values and label
# extracted target and features, where damage_grade is the target
import pandas as pd
train_values= pd.read_csv('../data/train_values.csv')

label=pd.read_csv('../data/train_labels.csv')
test_features=pd.read_csv('../data/test_values.csv')

merged_data = pd.merge(train_values, label, on=["building_id"])
target = merged_data.loc[:, "damage_grade"]
features = merged_data.drop(columns="damage_grade")


#roof_type=3
#ground_floor_type=5
#other_floor_type=4
#position=4
#plan_configuration=10
#legal_ownership_status=4

In [ ]:
#We encoded all categorical values, the geolocation and plan_configuration with JamesSteinerEncoding, the other ones with OneHotEncoding
#With JamesSteinerEncoding we were able to account for different sample sizes in each geolocation area. we also encoded plan_configuration 
#just for conveniance and we didn't want to OneHotEncode it
#We applied it to the encoded training as well to the encoded test set
import category_encoders as ce
encoder = ce.JamesSteinEncoder(cols=['geo_level_1_id','geo_level_2_id','geo_level_3_id','plan_configuration'])
encoder.fit(features, target)

X_encoded = encoder.transform(features)
X_test_encoded=encoder.transform(test_features)
encoder=ce.OneHotEncoder(cols=['land_surface_condition','foundation_type',\
                               'roof_type','ground_floor_type','other_floor_type','position','legal_ownership_status'])
encoder.fit(X_encoded)
X_encoded = encoder.transform(X_encoded)
X_test_encoded = encoder.transform(X_test_encoded)

In [ ]:
#We applied RandomForrestClassifier to the training set...
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=42, oob_score=True, n_estimators=180, max_depth=7)
fitted_model = rfc.fit(X_cleaned, target)
rfc.oob_score_


In [ ]:
# and predicted the the damage with the test set.
damage_grade_pred=rfc.predict(X_test_cleaned)

In [ ]:
#get building_id and damage_grade into the same dataframe
result = pd.DataFrame(predictions, columns=['predictions'])

result = X_test_cleaned.assign(damage_grade=damage_grade_pred)
prediction_df = result.loc[:,['building_id', 'damage_grade']]
prediction_df


In [ ]:
# Write csv-file
from pathlib import Path
filepath = Path('third_submit.csv')
prediction_df.to_csv(filepath, index=False)